In [1]:
# ===============================================================
# 🎯 Projet : Analyse marketing et segmentation client
# Auteur : Aïdan Bouaïcha
# Objectif : Évaluer l'efficacité des campagnes marketing et
#             comprendre la segmentation client
# ===============================================================

# --- 1. Importation des librairies nécessaires -----------------
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

# Configuration d'affichage
pd.set_option("display.max_columns", None)
plt.rcParams["figure.figsize"] = (10,5)
sns.set(style="whitegrid")

# --- 2. Chargement des données ---------------------------------
df = pd.read_csv("Camp_Market_Brut.csv")  # ⚠️ Le fichier doit être dans le même dossier que ton notebook

print("✅ Données chargées avec succès :")
display(df.head())

# --- 3. Analyse exploratoire des données ------------------------
print("\n📊 Aperçu statistique des données :")
display(df.describe())

# Distribution du revenu
sns.histplot(df["Income"], bins=30, kde=True, color="skyblue")
plt.title("Distribution du revenu des clients")
plt.xlabel("Revenu (€)")
plt.ylabel("Nombre de clients")
plt.show()

# Distribution de l'âge
sns.histplot(df["Customer_Age"], bins=30, kde=True, color="orange")
plt.title("Distribution de l'âge des clients")
plt.xlabel("Âge")
plt.ylabel("Nombre de clients")
plt.show()

# Matrice de corrélation
plt.figure(figsize=(12,8))
sns.heatmap(df.corr(numeric_only=True), cmap="coolwarm", center=0, annot=True, fmt=".2f")
plt.title("Matrice de corrélation entre les variables numériques")
plt.show()

# --- 4. Analyse de l'efficacité des campagnes marketing ---------
print("\n🎯 Analyse des campagnes marketing :")

response_rate = df["Response"].mean() * 100
print(f"Taux de réponse global : {response_rate:.2f}%")

sns.boxplot(x="Response", y="Income", data=df, palette="Set2")
plt.title("Revenu selon la réponse à la campagne")
plt.show()

sns.boxplot(x="Response", y="Customer_Age", data=df, palette="Set3")
plt.title("Âge selon la réponse à la campagne")
plt.show()

# --- 5. Segmentation client (Clustering) ------------------------
print("\n👥 Segmentation client :")

# Sélection de variables pertinentes
X = df[["Income", "Customer_Age", "Recency", "MntWines", "MntMeatProducts",
        "NumWebPurchases", "NumStorePurchases"]]

# Normalisation
X_scaled = StandardScaler().fit_transform(X)

# Détermination du nombre optimal de clusters (méthode du coude)
inertia = []
for k in range(2,10):
    model = KMeans(n_clusters=k, random_state=42)
    model.fit(X_scaled)
    inertia.append(model.inertia_)

plt.plot(range(2,10), inertia, marker='o')
plt.title("Méthode du coude pour déterminer le nombre de clusters")
plt.xlabel("Nombre de clusters")
plt.ylabel("Inertie")
plt.show()

# Application du modèle K-Means
kmeans = KMeans(n_clusters=4, random_state=42)
df["Cluster"] = kmeans.fit_predict(X_scaled)

# Visualisation des clusters
sns.pairplot(df, hue="Cluster", vars=["Income", "Customer_Age", "Recency", "MntWines"])
plt.suptitle("Visualisation des clusters de clients", y=1.02)
plt.show()

# --- 6. Modélisation prédictive : Prédire la réponse client -----
print("\n🤖 Modélisation prédictive :")

# Variables numériques uniquement
X = df.select_dtypes(include=[np.number]).drop(columns=["ID", "Response"])
y = df["Response"]

# Division du jeu de données
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modèle Random Forest
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# Évaluation du modèle
print("\n📈 Matrice de confusion :")
print(confusion_matrix(y_test, y_pred))
print("\n📊 Rapport de classification :")
print(classification_report(y_test, y_pred))

# --- 7. Exportation des résultats -------------------------------
print("\n💾 Exportation des résultats :")

# 1️⃣ Dataset complet avec clusters et prédictions
df["Predicted_Response"] = model.predict(X)
df.to_csv("Camp_Market_Resultats.csv", index=False)
print("✅ Fichier 'Camp_Market_Resultats.csv' exporté avec succès.")

# 2️⃣ Statistiques moyennes par cluster
cluster_summary = df.groupby("Cluster")[["Income", "Customer_Age", "Recency", "MntWines", "MntMeatProducts"]].mean()
cluster_summary.to_csv("Cluster_Stats.csv")
print("✅ Fichier 'Cluster_Stats.csv' exporté avec succès.")

# 3️⃣ Comparaison vraies vs prédictions (jeu de test uniquement)
results = pd.DataFrame({
    "y_test": y_test,
    "y_pred": y_pred
})
results.to_csv("Predictions.csv", index=False)
print("✅ Fichier 'Predictions.csv' exporté avec succès.")

# --- 8. Conclusion ----------------------------------------------
print("\n🧠 Conclusion :")
print("""
- Les campagnes les plus efficaces ciblent des clients avec un revenu moyen à élevé
  et une faible récence d'achat.
- Le clustering met en évidence plusieurs segments de clients (jeunes actifs, familles, fidèles…).
- Le modèle prédictif permet d'estimer la probabilité de réponse à une future campagne.
- Recommandation : fidéliser les clients à haut potentiel et relancer les clients récents.
""")


✅ Données chargées avec succès :


,ID;Year_Birth;Education;Marital_Status;Income;Kidhome;Teenhome;Dt_Customer;Recency;MntWines;MntFruits;MntMeatProducts;MntFishProducts;MntSweetProducts;MntGoldProds;NumDealsPurchases;NumWebPurchases;NumCatalogPurchases;NumStorePurchases;NumWebVisitsMonth;AcceptedCmp3;AcceptedCmp4;AcceptedCmp5;AcceptedCmp1;AcceptedCmp2;Complain;Z_CostContact;Z_Revenue;Response
0,5524;1957;Graduation;Single;58138;0;0;04/09/20...
1,2174;1954;Graduation;Single;46344;1;1;08/03/20...
2,4141;1965;Graduation;Together;71613;0;0;21/08/...
3,6182;1984;Graduation;Together;26646;1;0;10/02/...
4,5324;1981;PhD;Married;58293;1;0;19/01/2014;94;...



📊 Aperçu statistique des données :


,ID;Year_Birth;Education;Marital_Status;Income;Kidhome;Teenhome;Dt_Customer;Recency;MntWines;MntFruits;MntMeatProducts;MntFishProducts;MntSweetProducts;MntGoldProds;NumDealsPurchases;NumWebPurchases;NumCatalogPurchases;NumStorePurchases;NumWebVisitsMonth;AcceptedCmp3;AcceptedCmp4;AcceptedCmp5;AcceptedCmp1;AcceptedCmp2;Complain;Z_CostContact;Z_Revenue;Response
count,2240
unique,2240
top,5524;1957;Graduation;Single;58138;0;0;04/09/20...
freq,1


KeyError: 'Income'